In [35]:
import joblib 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.svm import SVR

In [26]:
X_train = joblib.load('X_train_transformed.pkl')
X_valid = joblib.load('X_valid_transformed.pkl')
X_test = joblib.load('X_test_transformed.pkl')

y_train = joblib.load('y_train.pkl')
y_valid = joblib.load('y_valid.pkl')
y_test = joblib.load('y_test.pkl')
transformer = joblib.load('transformer.pkl')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9246, 77) (1156, 77) (9246,) (1156,)


In [28]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)


print("Basic Linear Regression MSE:", mse_lr); print("Basic Linear Regression R2:", r2_lr)

Basic Linear Regression MSE: 95.31188464577222
Basic Linear Regression R2: 0.014760718894951785


In [29]:
param_grid_poly = {'kernel': ['polynomial'], 'degree': [2, 3, 4], 'alpha': [0.1, 1, 10]}

krr_poly_grid = GridSearchCV(KernelRidge(), param_grid_poly, cv=3, scoring='neg_mean_squared_error'); krr_poly_grid.fit(X_valid, y_valid)
best_params_poly = krr_poly_grid.best_params_
best_krr_poly = KernelRidge(**best_params_poly); best_krr_poly.fit(X_train, y_train)

y_pred_krr_poly = best_krr_poly.predict(X_test)
mse_krr_poly = mean_squared_error(y_test, y_pred_krr_poly); r2_krr_poly = r2_score(y_test, y_pred_krr_poly)

print("Best params for KRR Polynomial:", best_params_poly); print("Kernel Ridge Regression (Polynomial) MSE:", mse_krr_poly); print("Kernel Ridge Regression (Polynomial) R2:", r2_krr_poly)

Best params for KRR Polynomial: {'alpha': 10, 'degree': 2, 'kernel': 'polynomial'}
Kernel Ridge Regression (Polynomial) MSE: 95.56678483703487
Kernel Ridge Regression (Polynomial) R2: 0.012125814736603568


In [30]:
param_grid_rbf = {
    'kernel': ['rbf'],
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10],
    'coef0': [0, 0.1, 1]
}

krr_rbf_grid = GridSearchCV(KernelRidge(), param_grid_rbf, cv=3, scoring='neg_mean_squared_error')

krr_rbf_grid.fit(X_valid, y_valid)

best_params_rbf = krr_rbf_grid.best_params_
best_krr_rbf = KernelRidge(**best_params_rbf)
best_krr_rbf.fit(X_train, y_train)

y_pred_krr_rbf = best_krr_rbf.predict(X_test)
mse_krr_rbf = mean_squared_error(y_test, y_pred_krr_rbf)
r2_krr_rbf = r2_score(y_test, y_pred_krr_rbf)
print("Best parameters for Kernel Ridge Regression (RBF):", best_params_rbf)
print("Kernel Ridge Regression (RBF) MSE:", mse_krr_rbf)
print("Kernel Ridge Regression (RBF) R2:", r2_krr_rbf)


Best parameters for Kernel Ridge Regression (RBF): {'alpha': 10, 'coef0': 0, 'gamma': 0.0001, 'kernel': 'rbf'}
Kernel Ridge Regression (RBF) MSE: 96.44069512794619
Kernel Ridge Regression (RBF) R2: 0.0030922010380840526


In [43]:
# 1. Basic SVM (Linear Kernel) - Regression
print("Training Basic SVM (Linear Kernel)...")
svm_basic = SVR(kernel='linear')
svm_basic.fit(X_train, y_train)

y_pred_basic = svm_basic.predict(X_test)
mse_basic = mean_squared_error(y_test, y_pred_basic)
r2_basic = r2_score(y_test, y_pred_basic)
print("Basic SVM (Linear) MSE:", mse_basic)
print("Basic SVM (Linear) R²:", r2_basic)

# 2. SVM with Polynomial Kernel (Kernel Trick) - Regression
print("\nTraining SVM with Polynomial Kernel...")
param_grid_poly = {
    'kernel': ['poly'],
    'degree': [2, 3],
    'C': [0.1, 1, 10],
    'coef0': [0, 1],
    'epsilon': [0.1, 0.5]
}

svm_poly_grid = GridSearchCV(SVR(), param_grid_poly, cv=2, scoring='neg_mean_squared_error')
svm_poly_grid.fit(X_valid, y_valid)

best_params_poly = svm_poly_grid.best_params_
print("Best Polynomial Kernel Params:", best_params_poly)
best_svm_poly = SVR(**best_params_poly)
best_svm_poly.fit(X_train, y_train)

y_pred_poly = best_svm_poly.predict(X_test)
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print("SVM (Polynomial Kernel) MSE:", mse_poly)
print("SVM (Polynomial Kernel) R²:", r2_poly)

# 3. SVM with RBF Kernel (Kernel Trick) - Regression
print("\nTraining SVM with RBF Kernel...")
param_grid_rbf = {
    'kernel': ['rbf'],
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.1, 1],
    'epsilon': [0.1, 0.5]
}

svm_rbf_grid = GridSearchCV(SVR(), param_grid_rbf, cv=2, scoring='neg_mean_squared_error')
svm_rbf_grid.fit(X_valid, y_valid)

best_params_rbf = svm_rbf_grid.best_params_
print("Best RBF Kernel Params:", best_params_rbf)
best_svm_rbf = SVR(**best_params_rbf)
best_svm_rbf.fit(X_train, y_train)

y_pred_rbf = best_svm_rbf.predict(X_test)
mse_rbf = mean_squared_error(y_test, y_pred_rbf)
r2_rbf = r2_score(y_test, y_pred_rbf)
print("SVM (RBF Kernel) MSE:", mse_rbf)
print("SVM (RBF Kernel) R²:", r2_rbf)

print("\nSummary of Model Performance:")
print(f"Basic SVM (Linear) MSE: {mse_basic:.4f}, R²: {r2_basic:.4f}")
print(f"SVM (Polynomial) MSE: {mse_poly:.4f}, R²: {r2_poly:.4f}")
print(f"SVM (RBF) MSE: {mse_rbf:.4f}, R²: {r2_rbf:.4f}")

Training Basic SVM (Linear Kernel)...
Basic SVM (Linear) MSE: 100.58736648647843
Basic SVM (Linear) R²: -0.03977195513134735

Training SVM with Polynomial Kernel...
Best Polynomial Kernel Params: {'C': 1, 'coef0': 0, 'degree': 2, 'epsilon': 0.5, 'kernel': 'poly'}
SVM (Polynomial Kernel) MSE: 101.62426073990372
SVM (Polynomial Kernel) R²: -0.0504903346138581

Training SVM with RBF Kernel...
Best RBF Kernel Params: {'C': 10, 'epsilon': 0.5, 'gamma': 1, 'kernel': 'rbf'}
SVM (RBF Kernel) MSE: 98.30144036885515
SVM (RBF Kernel) R²: -0.016142328950345064

Summary of Model Performance:
Basic SVM (Linear) MSE: 100.5874, R²: -0.0398
SVM (Polynomial) MSE: 101.6243, R²: -0.0505
SVM (RBF) MSE: 98.3014, R²: -0.0161


In [44]:
print("Training Decision Tree Regressor...")
param_grid_dt = {
    'max_depth': [3, 5, 10, None],          # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],        # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4]           # Minimum samples at a leaf node
}

dt_grid = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_dt, cv=3, scoring='neg_mean_squared_error')
dt_grid.fit(X_valid, y_valid)  # Tune on validation set

best_params_dt = dt_grid.best_params_
print("Best Decision Tree Params:", best_params_dt)
best_dt = DecisionTreeRegressor(**best_params_dt, random_state=42)
best_dt.fit(X_train, y_train)

y_pred_dt = best_dt.predict(X_test)
mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)
print("Decision Tree MSE:", mse_dt)
print("Decision Tree R²:", r2_dt)

print("\nTraining Random Forest Regressor...")
param_grid_rf = {
    'n_estimators': [50, 100, 200],         # Number of trees
    'max_depth': [5, 10, None],             # Maximum depth of each tree
    'min_samples_split': [2, 5],            # Minimum samples to split a node
    'min_samples_leaf': [1, 2]              # Minimum samples at a leaf node
}

rf_grid = GridSearchCV(RandomForestRegressor(random_state=42, n_jobs=-1), param_grid_rf, cv=3, scoring='neg_mean_squared_error')
rf_grid.fit(X_valid, y_valid)

best_params_rf = rf_grid.best_params_
print("Best Random Forest Params:", best_params_rf)
best_rf = RandomForestRegressor(**best_params_rf, random_state=42, n_jobs=-1)
best_rf.fit(X_train, y_train)

y_pred_rf = best_rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest MSE:", mse_rf)
print("Random Forest R²:", r2_rf)

baseline_mse = mean_squared_error(y_test, np.full_like(y_test, y_train.mean()))
print("\nBaseline MSE (mean prediction):", baseline_mse)

print("\nSummary of Model Performance:")
print(f"Decision Tree MSE: {mse_dt:.4f}, R²: {r2_dt:.4f}")
print(f"Random Forest MSE: {mse_rf:.4f}, R²: {r2_rf:.4f}")

Training Decision Tree Regressor...
Best Decision Tree Params: {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10}
Decision Tree MSE: 95.26156791242974
Decision Tree R²: 0.015280843141473155

Training Random Forest Regressor...
Best Random Forest Params: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Random Forest MSE: 95.18085905916578
Random Forest R²: 0.016115130836697245

Baseline MSE (mean prediction): 97.030276816609

Summary of Model Performance:
Decision Tree MSE: 95.2616, R²: 0.0153
Random Forest MSE: 95.1809, R²: 0.0161


In [ ]:
class PlayOutcomeNet(nn.Module):
    def __init__(self, input_size):
        super(PlayOutcomeNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

random_seeds = [102, 203, 305, 405, 506, 607]
batch_size = 32

train_loaders = {}
valid_loaders = {}
test_loaders = {}

for seed in random_seeds:
    X_train_tensor = torch.load(f'X_train_seed{seed}.pt')
    y_train_tensor = torch.load(f'y_train_seed{seed}.pt')
    X_valid_tensor = torch.load(f'X_valid_seed{seed}.pt')
    y_valid_tensor = torch.load(f'y_valid_seed{seed}.pt')
    X_test_tensor = torch.load(f'X_test_seed{seed}.pt')
    y_test_tensor = torch.load(f'y_test_seed{seed}.pt')

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loaders[seed] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loaders[seed] = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loaders[seed] = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Seed {seed} - Train batches: {len(train_loaders[seed])}, "
          f"Valid batches: {len(valid_loaders[seed])}, Test batches: {len(test_loaders[seed])}")

num_epochs = 100
results = {'mse': [], 'r2': []}

for seed in random_seeds:
    print(f"\nTraining neural net with seed {seed}...")
    torch.manual_seed(seed)
    model = PlayOutcomeNet(input_size=X_train_tensor.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

    best_val_loss = float('inf')
    patience = 10
    counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loaders[seed]:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(valid_loaders[seed].dataset.tensors[0])
            val_loss = criterion(val_outputs, valid_loaders[seed].dataset.tensors[1])
        
        if (epoch + 1) % 20 == 0:
            print(f"Seed {seed}, Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {running_loss/len(train_loaders[seed]):.4f}, Val Loss: {val_loss.item():.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            counter = 0
        else:
            counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1} for seed {seed}")
            model.load_state_dict(best_model_state)
            break

    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, targets in test_loaders[seed]:
            outputs = model(inputs)
            y_pred.extend(outputs.numpy().flatten())
            y_true.extend(targets.numpy().flatten())
    
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    results['mse'].append(mse)
    results['r2'].append(r2)
    print(f"Seed {seed} - Test MSE: {mse:.4f}, Test R²: {r2:.4f}")

avg_mse = np.mean(results['mse'])
avg_r2 = np.mean(results['r2'])
print(f"\nNeural Network Average across 6 seeds - MSE: {avg_mse:.4f}, R²: {avg_r2:.4f}")

Seed 102 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 203 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 305 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 405 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 506 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 607 - Train batches: 289, Valid batches: 37, Test batches: 37

Training neural net with seed 102...
Seed 102, Epoch [20/100], Train Loss: 79.9207, Val Loss: 79.6967
Seed 102, Epoch [40/100], Train Loss: 78.3902, Val Loss: 79.1503
Early stopping at epoch 49 for seed 102
Seed 102 - Test MSE: 97.4210, Test R²: -0.0070

Training neural net with seed 203...
Seed 203, Epoch [20/100], Train Loss: 79.5282, Val Loss: 83.0422
Seed 203, Epoch [40/100], Train Loss: 78.6688, Val Loss: 82.7202
Early stopping at epoch 47 for seed 203
Seed 203 - Test MSE: 91.1573, Test R²: -0.0103

Training neural net with seed 305...
Seed 305, Epoch [20/100], Train Loss: 79.1893, Val

In [33]:
class PlayOutcomeNet(nn.Module):
    def __init__(self, input_size):
        super(PlayOutcomeNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

random_seeds = [102, 203, 305, 405, 506, 607]
batch_size = 32

train_loaders = {}
valid_loaders = {}
test_loaders = {}

for seed in random_seeds:
    X_train_tensor = torch.load(f'X_train_seed{seed}.pt')
    y_train_tensor = torch.load(f'y_train_seed{seed}.pt')
    X_valid_tensor = torch.load(f'X_valid_seed{seed}.pt')
    y_valid_tensor = torch.load(f'y_valid_seed{seed}.pt')
    X_test_tensor = torch.load(f'X_test_seed{seed}.pt')
    y_test_tensor = torch.load(f'y_test_seed{seed}.pt')

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loaders[seed] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loaders[seed] = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loaders[seed] = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Seed {seed} - Train batches: {len(train_loaders[seed])}, "
          f"Valid batches: {len(valid_loaders[seed])}, Test batches: {len(test_loaders[seed])}")

num_epochs = 100
results = {'mse': [], 'r2': []}

for seed in random_seeds:
    print(f"\nTraining neural net with seed {seed}...")
    torch.manual_seed(seed)
    model = PlayOutcomeNet(input_size=X_train_tensor.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loaders[seed]:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(valid_loaders[seed].dataset.tensors[0])
            val_loss = criterion(val_outputs, valid_loaders[seed].dataset.tensors[1])
        
        if (epoch + 1) % 20 == 0:
            print(f"Seed {seed}, Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {running_loss/len(train_loaders[seed]):.4f}, Val Loss: {val_loss.item():.4f}")

    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, targets in test_loaders[seed]:
            outputs = model(inputs)
            y_pred.extend(outputs.numpy().flatten())
            y_true.extend(targets.numpy().flatten())
    
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    results['mse'].append(mse)
    results['r2'].append(r2)
    print(f"Seed {seed} - Test MSE: {mse:.4f}, Test R²: {r2:.4f}")


avg_mse = np.mean(results['mse'])
avg_r2 = np.mean(results['r2'])
print(f"\nNeural Network Average across 6 seeds - MSE: {avg_mse:.4f}, R²: {avg_r2:.4f}")

Seed 102 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 203 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 305 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 405 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 506 - Train batches: 289, Valid batches: 37, Test batches: 37
Seed 607 - Train batches: 289, Valid batches: 37, Test batches: 37

Training neural net with seed 102...
Seed 102, Epoch [20/100], Train Loss: 79.9207, Val Loss: 79.6967
Seed 102, Epoch [40/100], Train Loss: 78.3902, Val Loss: 79.1503
Seed 102, Epoch [60/100], Train Loss: 76.7471, Val Loss: 79.3862
Seed 102, Epoch [80/100], Train Loss: 76.7021, Val Loss: 79.8244
Seed 102, Epoch [100/100], Train Loss: 76.1161, Val Loss: 80.2313
Seed 102 - Test MSE: 97.2482, Test R²: -0.0053

Training neural net with seed 203...
Seed 203, Epoch [20/100], Train Loss: 79.5282, Val Loss: 83.0422
Seed 203, Epoch [40/100], Train Loss: 78.6688, Val Loss: 82.7202
Seed 203, Epoch [60